## 📌 Project Overview

This project focuses on building a Named Entity Recognition (NER) system for Persian (Farsi) text 

by fine-tuning a transformer-based model on the WikiAnn-fa dataset.

The goal is to automatically identify and extract named entities such as persons (PER), locations 

(LOC), and organizations (ORG) from Persian text.

The model is trained using Hugging Face Transformers and PyTorch, and the final system supports 

both offline inference and deployment as a RESTful API using FastAPI.

This cell installs the required Python libraries for training and evaluating the NER model:
- `datasets` for loading and processing the WikiAnn-fa dataset
- `transformers` for fine-tuning a pretrained Transformer model
- `evaluate` and `seqeval` for computing NER sequence-labeling metrics (e.g., precision, recall, F1)


In [ ]:
! pip install evaluate seqeval datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=3d7723b8f859776f97444a7de8be9ee93ec3661e8d1abc90a5803f0ad7b98413
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


This cell imports the core libraries required for the project, including PyTorch for model training, Hugging Face Transformers for token classification (NER), Datasets for loading the WikiAnn-fa dataset, and evaluation utilities for computing NER metrics. It also includes supporting libraries for numerical operations and configuration handling.


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
import evaluate
import json

This cell loads the Persian (Farsi) Named Entity Recognition dataset from WikiAnn using the Hugging Face Datasets library. The dataset provides token-level annotations for entities such as persons, locations, and organizations, and is used for training, validation, and evaluation of the NER model.


In [5]:
# dataset = load_dataset("AliFartout/PEYMA-ARMAN-Mixed")
dataset = load_dataset('wikiann', 'fa')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

fa/validation-00000-of-00001.parquet:   0%|          | 0.00/603k [00:00<?, ?B/s]

fa/test-00000-of-00001.parquet:   0%|          | 0.00/597k [00:00<?, ?B/s]

fa/train-00000-of-00001.parquet:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [6]:
print(dataset)
print(dataset['train'][0])

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 20000
    })
})
{'tokens': ['تغییرمسیر', 'مهتر', '(', 'خرم\u200cآباد', ')'], 'ner_tags': [0, 5, 6, 6, 6], 'langs': ['fa', 'fa', 'fa', 'fa', 'fa'], 'spans': ['LOC: مهتر ( خرم\u200cآباد )']}


This cell initializes the tokenizer from the pretrained ParsBERT model. The tokenizer is responsible for converting Persian text into token IDs and subword representations that can be processed by the transformer-based NER model.


In [7]:
model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

This cell defines a preprocessing function that tokenizes the input tokens and aligns the original NER labels with the subword tokens produced by the tokenizer. Special tokens and non-initial subword pieces are assigned a label of `-100` so they are ignored during loss computation.


In [8]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


This cell applies the tokenization and label-alignment function to the entire dataset. It processes the data in batches and removes the original columns, producing tokenized datasets that are ready to be used for training and evaluation.


In [9]:
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

This cell inspects the first sample from the tokenized training dataset to verify that the input tokens, attention masks, and aligned NER labels have been processed correctly.


In [10]:
tokenized_datasets['train'][0]

{'input_ids': [2, 2671, 85815, 61044, 9, 19530, 10, 4],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 0, -100, 5, 6, 6, 6, -100]}

This cell extracts the list of NER label names from the training dataset and computes the total number of unique labels. These labels are later used to configure the token classification model and to correctly interpret the model’s predictions.


In [11]:
label_list = dataset["train"].features["ner_tags"].feature.names
num_labels = len(label_list)

print("Labels:", label_list)
print("Num labels:", num_labels)

Labels: ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
Num labels: 7


This cell initializes a transformer-based token classification model using the pretrained ParsBERT checkpoint. It configures the model with the correct number of NER labels and defines mappings between label IDs and label names, enabling proper training and human-readable predictions.


In [12]:
model = AutoModelForTokenClassification.from_pretrained(
    pretrained_model_name_or_path=model_checkpoint,
    num_labels = num_labels,
    id2label = {i:l for i, l in enumerate(label_list)},
    label2id = {l:i for i, l in enumerate(label_list)}
)

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

This cell creates a data collator specifically designed for token classification tasks. It dynamically pads input sequences and their corresponding labels to the same length within each batch, ensuring efficient and correct batching during training.


In [14]:
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True)


In [15]:
device = 'cuda' if torch.cuda.is_available else 'mps' if torch.mps.is_available else 'cpu'
print(device)

cuda


This cell defines the training configuration using Hugging Face `TrainingArguments`. It specifies key hyperparameters such as learning rate, batch size, number of epochs, evaluation and checkpointing strategy, and enables mixed-precision (FP16) training to improve performance and reduce memory usage.


In [16]:
args = TrainingArguments(
    output_dir="./ner-wikiann-fa",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_steps=50,
    save_total_limit=2,
    report_to="none",
    fp16 = True
)

In [17]:
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

This cell defines the evaluation metric for the NER task using `seqeval`. It converts model outputs into label predictions, filters out ignored tokens (`-100`), and computes standard NER metrics including precision, recall, F1-score, and accuracy.


In [18]:
metric = evaluate.load("seqeval")

def compute_metrics(p):

    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    true_predictions = [[label_list[p] for (p, l) in zip(pred, lab) if l != -100] for (pred , lab) in zip(predictions, labels)]

    true_labels = [[label_list[l] for (p, l) in zip(pred, lab) if l != -100] for (pred, lab) in zip(predictions, labels)]


    results = metric.compute(
        predictions = true_predictions,
        references = true_labels
    )

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

This cell initializes the Hugging Face `Trainer` with the model, training arguments, datasets, tokenizer, and evaluation function. It then starts the fine-tuning process on the Persian NER dataset, performing training and evaluation at the end of each epoch.


In [19]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


/tmp/ipython-input-4140853055.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.134900,0.141016,0.899631,0.919750,0.909579,0.962320
2,0.094200,0.126547,0.920164,0.928124,0.924127,0.970460
3,0.053800,0.145281,0.934763,0.938892,0.936823,0.972779
4,0.022900,0.160565,0.930345,0.941561,0.935919,0.971745
5,0.026700,0.164885,0.935195,0.942941,0.939052,0.972945


TrainOutput(global_step=6250, training_loss=0.07728475940465927, metrics={'train_runtime': 720.416, 'train_samples_per_second': 138.809, 'train_steps_per_second': 8.676, 'total_flos': 1016955787648416.0, 'train_loss': 0.07728475940465927, 'epoch': 5.0})

This cell evaluates the fine-tuned NER model on the validation dataset and reports performance metrics such as precision, recall, F1-score, and accuracy.


In [20]:
trainer.evaluate()


{'eval_loss': 0.16488535702228546,
 'eval_precision': 0.9351953267615918,
 'eval_recall': 0.9429412847413952,
 'eval_f1': 0.9390523325084776,
 'eval_accuracy': 0.9729454739545977,
 'eval_runtime': 10.6831,
 'eval_samples_per_second': 936.054,
 'eval_steps_per_second': 58.503,
 'epoch': 5.0}

This cell evaluates the trained NER model on different dataset splits. It reports performance metrics for the training set, validation set, and test set, allowing a comprehensive comparison of the model’s generalization performance.


In [32]:
train_evaluate = trainer.evaluate()
validation_evaluate = trainer.evaluate(eval_dataset=tokenized_datasets['validation'])
test_evaluate = trainer.evaluate(eval_dataset=tokenized_datasets['test'])

In [35]:
train_evaluate

{'eval_loss': 0.16488535702228546,
 'eval_precision': 0.9351953267615918,
 'eval_recall': 0.9429412847413952,
 'eval_f1': 0.9390523325084776,
 'eval_accuracy': 0.9729454739545977,
 'eval_runtime': 12.0197,
 'eval_samples_per_second': 831.967,
 'eval_steps_per_second': 51.998,
 'epoch': 5.0}

In [33]:
validation_evaluate

{'eval_loss': 0.16488535702228546,
 'eval_precision': 0.9351953267615918,
 'eval_recall': 0.9429412847413952,
 'eval_f1': 0.9390523325084776,
 'eval_accuracy': 0.9729454739545977,
 'eval_runtime': 11.6282,
 'eval_samples_per_second': 859.981,
 'eval_steps_per_second': 53.749,
 'epoch': 5.0}

In [34]:
test_evaluate

{'eval_loss': 0.1793498992919922,
 'eval_precision': 0.9383161989441512,
 'eval_recall': 0.9431204617389686,
 'eval_f1': 0.9407121964808024,
 'eval_accuracy': 0.9720461918609538,
 'eval_runtime': 11.3411,
 'eval_samples_per_second': 881.751,
 'eval_steps_per_second': 55.109,
 'epoch': 5.0}

This cell aggregates the evaluation results from the training, validation, and test sets into a single dictionary and saves them as a CSV file. This allows the model’s performance metrics to be easily reviewed, compared, and reused for reporting or documentation.


In [36]:
all_evaluate = {
    'train': train_evaluate,
    'validation': validation_evaluate,
    'test': test_evaluate
}
pd.DataFrame(all_evaluate).to_csv('results.csv')

In [2]:
import pandas as pd
pd.read_csv('./results.csv')

,Unnamed: 0,train,validation,test
0,eval_loss,0.164885,0.164885,0.179350
1,eval_precision,0.935195,0.935195,0.938316
2,eval_recall,0.942941,0.942941,0.943120
3,eval_f1,0.939052,0.939052,0.940712
4,eval_accuracy,0.972945,0.972945,0.972046
5,eval_runtime,12.019700,11.628200,11.341100
6,eval_samples_per_second,831.967000,859.981000,881.751000
7,eval_steps_per_second,51.998000,53.749000,55.109000
8,epoch,5.000000,5.000000,5.000000


This cell saves the fine-tuned NER model, tokenizer, and label mappings to disk for later inference or deployment. It also packages the saved files into a ZIP archive and uploads them to Google Drive, ensuring the trained model is safely stored and easily transferable.


In [ ]:
model_save_path = "ner_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

with open(f"{model_save_path}/labels.json", "w", encoding="utf-8") as f:
    json.dump(label_list, f, ensure_ascii=False)


notebook_name = "model_ner_wikiann-fa.ipynb"
!cp "{notebook_name}" ner_model/


!zip -r ner_model.zip ner_model


from google.colab import drive
drive.mount('/content/drive')

# کپی به درایو
!cp ner_model.zip /content/drive/MyDrive/


cp: cannot stat 'model_ner_wikiann-fa.ipynb': No such file or directory
  adding: ner_model/ (stored 0%)
  adding: ner_model/vocab.txt (deflated 62%)
  adding: ner_model/special_tokens_map.json (deflated 42%)
  adding: ner_model/config.json (deflated 52%)
  adding: ner_model/labels.json (deflated 39%)
  adding: ner_model/tokenizer.json (deflated 72%)
  adding: ner_model/tokenizer_config.json (deflated 74%)
  adding: ner_model/model.safetensors (deflated 8%)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
# Save the current notebook to Google Drive (manually)
from google.colab import files
files.download('ner_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

This cell loads the saved NER model, tokenizer, and label mappings from disk and defines an inference function for Persian text. The function tokenizes the input sentence, runs the model in inference mode, aligns predictions with original words, and prints the recognized named entities for each token.


In [5]:
import json

model_dir = "./ner_model"

# tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForTokenClassification.from_pretrained(model_dir)


with open(f"{model_dir}/labels.json", "r", encoding="utf-8") as f:
    label_list = json.load(f)


def predict_ner(sentence):
    tokens = sentence.split()
    inputs = tokenizer(tokens, is_split_into_words=True, return_tensors="pt", truncation=True)
    word_ids = inputs.word_ids()

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)[0].numpy()


    final_tokens, final_tags = [], []
    prev_word_id = None
    for i, word_id in enumerate(word_ids):
        if word_id is None or word_id == prev_word_id:
            continue
        final_tokens.append(tokens[word_id])
        final_tags.append(label_list[predictions[i]])
        prev_word_id = word_id

    for token, tag in zip(final_tokens, final_tags):
        print(f"{token:10} → {tag}")

predict_ner("علی دیروز به سازمان ملل متحد در نیویورک رفت")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


علی        → B-PER
دیروز      → O
به         → O
سازمان     → B-ORG
ملل        → I-ORG
متحد       → I-ORG
در         → O
نیویورک    → B-LOC
رفت        → O


In [6]:
predict_ner("«محمدرضا شریفی‌نیا در سال ۱۳۹۸ به همراه تیم تحقیقات انرژی دانشگاه صنعتی شریف، سفری به آلمان داشت و در کنفرانسی که در شهر برلین توسط شرکت زیمنس برگزار شد، درباره نقش ایران در بازار جهانی گاز سخنرانی کرد.»")

«محمدرضا   → O
شریفی‌نیا  → I-PER
در         → O
سال        → O
۱۳۹۸       → O
به         → O
همراه      → O
تیم        → O
تحقیقات    → O
انرژی      → O
دانشگاه    → B-ORG
صنعتی      → I-ORG
شریف،      → I-ORG
سفری       → O
به         → O
آلمان      → B-LOC
داشت       → O
و          → O
در         → O
کنفرانسی   → O
که         → O
در         → O
شهر        → O
برلین      → B-LOC
توسط       → O
شرکت       → O
زیمنس      → O
برگزار     → O
شد،        → O
درباره     → O
نقش        → O
ایران      → B-LOC
در         → O
بازار      → O
جهانی      → I-ORG
گاز        → O
سخنرانی    → O
کرد.»      → O
